In [1]:
import pandas as pd
import numpy as np
import os

# Loading Data

In [2]:
properties_df = pd.read_csv(os.getcwd() + '/DATA/properties.csv')
transactions_df = pd.read_csv(os.getcwd() + '/DATA/transactions.csv')

transactions_df['SALE_DATE'] = pd.to_datetime(transactions_df['SALE_DATE'], dayfirst=True)

df = properties_df.merge(transactions_df, how='left', on='ID')
df.head()

,ID,PROPERTYCATEGORY,STATE,POSTCODE,SUBURB,STREETTYPE,ADDRESSLATITUDE,ADDRESSLONGITUDE,AREASIZE,BEDROOMS,BATHS,PARKING,AIRCONDITION,BALCONY,WARDROBE,GARDEN,SALE_DATE,SALE_PRICE
0,GL-3704-XQ,Unit,NSW,2042.0,B,St,-33.90505,151.17824,4120.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaT,NaN
1,JE-8988-FK,House,NSW,2042.0,B,Rd,-33.89965,151.17771,42.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN
2,MH-4739-NW,House,NSW,2035.0,A,Rd,-33.94226,151.26235,748.0,3.0,1.0,1,False,False,False,True,NaT,NaN
3,SF-3867-VO,House,NSW,2042.0,B,St,-33.90520,151.18065,72.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN
4,EP-8786-PA,Unit,NSW,2035.0,A,St,-33.94649,151.25793,1561.0,2.0,2.0,0,False,True,False,False,2012-07-06,630000.0


# Data Cleaning

### Filtering for only houses sold in 2020.

In [3]:
df_year = df.loc[df['SALE_DATE'].dt.year == 2020]

### Checking for unique sales to ensure no properties are double counted for multiple sales in one year.

In [4]:
unique_sales = df_year[['ID', "SALE_DATE"]].groupby("ID").count()
more_than_one_sale_2020 = df_year.loc[df_year['ID'] == unique_sales.loc[unique_sales['SALE_DATE'] > 1].index[0]]
print(f"***Property: {unique_sales.loc[unique_sales['SALE_DATE'] > 1].index[0]} "
      f"was sold more than once in 2020****")

***Property: RE-6162-BJ was sold more than once in 2020****


### ASSUMPTION 1: Where a property has been sold more than once during the year. We will only include
### their most recent sales price.


In [5]:
df_year = df_year.sort_values(by='SALE_DATE').drop_duplicates(subset= ['ID'], keep='last')

# Results

### Agregating by Suburb to get the total values as deterimined by sales price.

In [9]:
phase1_result = df_year.groupby("SUBURB").agg({'SALE_PRICE': ['sum']}).reset_index()
display(phase1_result)

,SUBURB,SALE_PRICE
,,sum
0,A,187191000.0
1,B,232930790.0


### Extracting our answer for phase 1

In [10]:
answers = phase1_result[phase1_result[('SALE_PRICE', 'sum')] ==phase1_result[('SALE_PRICE', 'sum')].max()].reset_index(drop=True)
print(f"Suburb {answers[('SUBURB', '')][0]} has the highest Total property market value in 2020"
      f" with a value of ${answers[('SALE_PRICE', 'sum')][0]}")

Suburb B has the highest Total property market value in 2020 with a value of $232930790.0
